In [36]:
#pip install tensorflow==2.14

In [37]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
train_dir='/content/drive/MyDrive/rini_datascience/emergency/emergency_less/train/images'
test_dir=('/content/drive/MyDrive/rini_datascience/emergency/emergency_less/train/images')
df_test=pd.read_csv('/content/drive/MyDrive/rini_datascience/emergency/emergency_less/train/train.csv')
df_train=pd.read_csv('/content/drive/MyDrive/rini_datascience/emergency/emergency_less/train/train.csv')

In [40]:
data = pd.read_csv('/content/drive/MyDrive/rini_datascience/emergency/emergency_less/train/train.csv', dtype = str)
train_df = data
# train_df["emergency_or_not"] = train_df["emergency_or_not"].replace({0: 'not_emergency', 1: 'emergency'})
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True,
    validation_split = 0.2)

In [41]:

training_set = train_datagen.flow_from_dataframe(
    train_df,
    "/content/drive/MyDrive/rini_datascience/emergency/emergency_less/train/images",
    x_col='image_names',
    y_col='emergency_or_not',
    target_size= (128,128),
    class_mode='binary',
    shuffle = True,
    batch_size = 8,
    subset = 'training')

Found 1317 validated image filenames belonging to 2 classes.


In [42]:
validation_set = train_datagen.flow_from_dataframe(
    train_df,
    "/content/drive/MyDrive/rini_datascience/emergency/emergency_less/train/images",
    x_col='image_names',
    y_col='emergency_or_not',
    target_size= (128,128),
    class_mode='binary',
    shuffle = True,
    batch_size = 8,
    subset = 'validation')

Found 329 validated image filenames belonging to 2 classes.


In [43]:

# Preparing test data
test_data = pd.read_csv('/content/drive/MyDrive/rini_datascience/emergency/emergency_less/test.csv')
test_df = test_data
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_dataframe(
        test_df,
        '/content/drive/MyDrive/rini_datascience/emergency/emergency_less/train/images',
        x_col = 'image_names',
        y_col = None,
        target_size=(128,128),
        batch_size=8,
        shuffle = False,
        class_mode = None)


Found 706 validated image filenames.


In [44]:
type(training_set)

keras.src.preprocessing.image.DataFrameIterator

In [45]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706 entries, 0 to 705
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_names  706 non-null    object
dtypes: object(1)
memory usage: 5.6+ KB


# MODEL BUILDING

## CNN

In [46]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[128,128, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

cnn.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dense_10 (Dense)            (None, 128)               7

In [47]:
#Early stopping to get opimal epochs
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

ES = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 8)
MC = ModelCheckpoint('best_cnn.h5', monitor = 'val_accuracy')

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
cnn.fit(x = training_set, validation_data = validation_set, epochs = 10, callbacks = [ES, MC])


Epoch 1/10
165/165 [==============================] - 38s 221ms/step - loss: 0.6619 - accuracy: 0.6591 - val_loss: 0.5380 - val_accuracy: 0.7842


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/10
165/165 [==============================] - 38s 229ms/step - loss: 0.5633 - accuracy: 0.7380 - val_loss: 0.5275 - val_accuracy: 0.7386
Epoch 3/10
165/165 [==============================] - 38s 230ms/step - loss: 0.5328 - accuracy: 0.7456 - val_loss: 0.5455 - val_accuracy: 0.7386
Epoch 4/10
165/165 [==============================] - 35s 215ms/step - loss: 0.5034 - accuracy: 0.7547 - val_loss: 0.5100 - val_accuracy: 0.7690
Epoch 5/10
165/165 [==============================] - 32s 196ms/step - loss: 0.4892 - accuracy: 0.7563 - val_loss: 0.5349 - val_accuracy: 0.7356
Epoch 6/10
165/165 [==============================] - 33s 199ms/step - loss: 0.4784 - accuracy: 0.7715 - val_loss: 0.5599 - val_accuracy: 0.8207
Epoch 7/10
165/165 [==============================] - 33s 198ms/step - loss: 0.4813 - accuracy: 0.7616 - val_loss: 0.4726 - val_accuracy: 0.7812
Epoch 8/10
165/165 [==============================] - 33s 199ms/step - loss: 0.4378 - accuracy: 0.7790 - val_loss: 0.4376 - val_ac

In [48]:
#prediction

import tensorflow as tf
from keras.preprocessing import image
import numpy as np

model = tf.keras.models.load_model('best_cnn.h5')


img_path = '/content/drive/MyDrive/rini_datascience/emergency/dataset/images/436.jpg'
img = image.load_img(img_path, target_size=(128,128,3))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255


predictions = cnn.predict(img)
predicted_class = np.argmax(predictions, axis=1)[0]
class_labels = ['non-emergency', 'emergency']
predicted_label = class_labels[predicted_class]
print('Predicted class:', predicted_label)


1/1 [==============================] - 0s 71ms/step
Predicted class: non-emergency


In [49]:
#evaluation
validation_loss, validation_accuracy = cnn.evaluate(validation_set)
print("Validation Accuracy:", validation_accuracy)



42/42 [==============================] - 4s 90ms/step - loss: 0.4582 - accuracy: 0.7994
Validation Accuracy: 0.7993921041488647


In [50]:
validation_loss

0.4581691324710846

## VGG16

In [51]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from keras.applications import VGG16

# Load pre-trained VGG16 model without the top layers
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(128,128,3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Adding dropout
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)  # Adding dropout
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model_vgg = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_vgg.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary
model_vgg.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                            

In [53]:
#Early stopping to get opimal epochs
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

ES = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 4)
MC = ModelCheckpoint('best_model1.h5', monitor = 'val_accuracy')

model_vgg.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model_vgg.fit(x = training_set, validation_data = validation_set, epochs = 10, callbacks = [ES, MC])

Epoch 1/10
165/165 [==============================] - 224s 1s/step - loss: 0.7360 - accuracy: 0.5793 - val_loss: 0.6112 - val_accuracy: 0.7234
Epoch 2/10
165/165 [==============================] - 219s 1s/step - loss: 0.6173 - accuracy: 0.6644 - val_loss: 0.5591 - val_accuracy: 0.7143
Epoch 3/10
165/165 [==============================] - 220s 1s/step - loss: 0.5812 - accuracy: 0.6811 - val_loss: 0.5112 - val_accuracy: 0.7842
Epoch 4/10
165/165 [==============================] - 260s 2s/step - loss: 0.5546 - accuracy: 0.7365 - val_loss: 0.4939 - val_accuracy: 0.7964
Epoch 5/10
165/165 [==============================] - 258s 2s/step - loss: 0.5277 - accuracy: 0.7487 - val_loss: 0.4695 - val_accuracy: 0.7994
Epoch 6/10
165/165 [==============================] - 258s 2s/step - loss: 0.5238 - accuracy: 0.7418 - val_loss: 0.4633 - val_accuracy: 0.8085
Epoch 7/10
165/165 [==============================] - 257s 2s/step - loss: 0.5141 - accuracy: 0.7578 - val_loss: 0.4673 - val_accuracy: 0.7933

In [55]:
#prediction

import tensorflow as tf
from keras.preprocessing import image
import numpy as np

model = tf.keras.models.load_model('best_model1.h5')


img_path = '/content/drive/MyDrive/rini_datascience/emergency/dataset/images/436.jpg'
img = image.load_img(img_path, target_size=(128,128,3))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255


predictions = model_vgg.predict(img)
predicted_class = np.argmax(predictions, axis=1)[0]
class_labels = ['non-emergency', 'emergency']
predicted_label = class_labels[predicted_class]
print('Predicted class:', predicted_label)


1/1 [==============================] - 0s 258ms/step
Predicted class: non-emergency


In [56]:
preds = model_vgg.predict(test_set)

89/89 [==============================] - 173s 2s/step


In [58]:
metrics=['accuracy',
tf.keras.metrics.Precision(),
tf.keras.metrics.Recall(),
tf.keras.metrics.AUC(),
tf.keras.metrics.SensitivityAtSpecificity(0.5),
tf.keras.metrics.SpecificityAtSensitivity(0.5)]

In [59]:
metrics

['accuracy',

In [60]:
preds

array([[0.35298583],
       [0.76595986],
       [0.0606507 ],
       [0.85399324],
       [0.07265988],
       [0.64716846],
       [0.42826378],
       [0.8816446 ],
       [0.4017273 ],
       [0.8192291 ],
       [0.14841573],
       [0.06936019],
       [0.27851096],
       [0.04736301],
       [0.1538136 ],
       [0.95577085],
       [0.16578898],
       [0.4213994 ],
       [0.83103824],
       [0.39453706],
       [0.7535531 ],
       [0.733003  ],
       [0.11744528],
       [0.08380769],
       [0.34069633],
       [0.15544036],
       [0.53800577],
       [0.17163636],
       [0.03107036],
       [0.45652837],
       [0.73067826],
       [0.08012361],
       [0.7434226 ],
       [0.08320235],
       [0.73730624],
       [0.09551719],
       [0.21256293],
       [0.9430753 ],
       [0.5861674 ],
       [0.15003282],
       [0.9055935 ],
       [0.5626119 ],
       [0.30638894],
       [0.52046883],
       [0.45619223],
       [0.7013897 ],
       [0.2645941 ],
       [0.179

In [61]:
#evaluation
validation_loss, validation_accuracy = model_vgg.evaluate(validation_set)
print("Validation Accuracy:", validation_accuracy)



42/42 [==============================] - 45s 1s/step - loss: 0.4335 - accuracy: 0.7903
Validation Accuracy: 0.7902735471725464


In [62]:
validation_loss

0.43353089690208435

## Xception

In [76]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception


In [77]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(128,128, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False


In [78]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 63, 63, 32)           864       ['input_8[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 63, 63, 32)           128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 63, 63, 32)           0         ['block1_conv1_bn[0][0]'

In [80]:
#Early stopping to get opimal epochs
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

ES = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 4)
MC = ModelCheckpoint('best_model3.h5', monitor = 'val_accuracy')

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(x = training_set, validation_data = validation_set, epochs = 10, callbacks = [ES, MC])

Epoch 1/10
165/165 [==============================] - 83s 486ms/step - loss: 0.4335 - accuracy: 0.8041 - val_loss: 0.3959 - val_accuracy: 0.8116
Epoch 2/10
165/165 [==============================] - 84s 507ms/step - loss: 0.3540 - accuracy: 0.8383 - val_loss: 0.3330 - val_accuracy: 0.8541
Epoch 3/10
165/165 [==============================] - 85s 513ms/step - loss: 0.3463 - accuracy: 0.8527 - val_loss: 0.3246 - val_accuracy: 0.8541
Epoch 4/10
165/165 [==============================] - 83s 505ms/step - loss: 0.3142 - accuracy: 0.8702 - val_loss: 0.3163 - val_accuracy: 0.8511
Epoch 5/10
165/165 [==============================] - 83s 506ms/step - loss: 0.3057 - accuracy: 0.8740 - val_loss: 0.3280 - val_accuracy: 0.8602
Epoch 6/10
165/165 [==============================] - 78s 475ms/step - loss: 0.2913 - accuracy: 0.8762 - val_loss: 0.3438 - val_accuracy: 0.8541
Epoch 7/10
165/165 [==============================] - 84s 509ms/step - loss: 0.2885 - accuracy: 0.8884 - val_loss: 0.3512 - val_ac

In [82]:
#prediction

import tensorflow as tf
from keras.preprocessing import image
import numpy as np

model = tf.keras.models.load_model('best_model3.h5')


img_path = '/content/drive/MyDrive/rini_datascience/emergency/dataset/images/436.jpg'
img = image.load_img(img_path, target_size=(128,128,3))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255


predictions = model.predict(img)
predicted_class = np.argmax(predictions, axis=1)[0]
class_labels = ['non-emergency', 'emergency']
predicted_label = class_labels[predicted_class]
print('Predicted class:', predicted_label)


1/1 [==============================] - 1s 670ms/step
Predicted class: non-emergency


In [83]:
#evaluation
validation_loss, validation_accuracy = model.evaluate(validation_set)
print("Validation Accuracy:", validation_accuracy)



42/42 [==============================] - 16s 373ms/step - loss: 0.2985 - accuracy: 0.8784
Validation Accuracy: 0.8784194588661194


In [84]:
validation_loss


0.2985162138938904

## DenseNet

In [85]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import DenseNet121


In [86]:
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(128,128, 3))
for layer in base_model.layers:
    layer.trainable = False

In [87]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 zero_padding2d_2 (ZeroPadd  (None, 134, 134, 3)          0         ['input_9[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv1/conv (Conv2D)         (None, 64, 64, 64)           9408      ['zero_padding2d_2[0][0]']    
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 64, 64, 64)           256       ['conv1/conv[0][0]']    

In [88]:
#Early stopping to get opimal epochs
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

ES = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 4)
MC = ModelCheckpoint('best_model4.h5', monitor = 'val_accuracy')

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(x = training_set, validation_data = validation_set, epochs = 10, callbacks = [ES, MC])

Epoch 1/10
165/165 [==============================] - 75s 420ms/step - loss: 0.5153 - accuracy: 0.7882 - val_loss: 0.3414 - val_accuracy: 0.8480
Epoch 2/10
165/165 [==============================] - 58s 352ms/step - loss: 0.3470 - accuracy: 0.8421 - val_loss: 0.3557 - val_accuracy: 0.8267
Epoch 3/10
165/165 [==============================] - 67s 406ms/step - loss: 0.3302 - accuracy: 0.8648 - val_loss: 0.3150 - val_accuracy: 0.8693
Epoch 4/10
165/165 [==============================] - 68s 410ms/step - loss: 0.3117 - accuracy: 0.8626 - val_loss: 0.2925 - val_accuracy: 0.8723
Epoch 5/10
165/165 [==============================] - 58s 350ms/step - loss: 0.2937 - accuracy: 0.8778 - val_loss: 0.2496 - val_accuracy: 0.8906
Epoch 6/10
165/165 [==============================] - 59s 355ms/step - loss: 0.2972 - accuracy: 0.8671 - val_loss: 0.2547 - val_accuracy: 0.8754
Epoch 7/10
165/165 [==============================] - 69s 420ms/step - loss: 0.2813 - accuracy: 0.8793 - val_loss: 0.2752 - val_ac

In [89]:
#evaluation
validation_loss, validation_accuracy = model.evaluate(validation_set)
print("Validation Accuracy:", validation_accuracy)



42/42 [==============================] - 11s 267ms/step - loss: 0.2355 - accuracy: 0.8967
Validation Accuracy: 0.8966565132141113


In [90]:
validation_loss

0.2355261743068695

In [91]:
#prediction

import tensorflow as tf
from keras.preprocessing import image
import numpy as np

model = tf.keras.models.load_model('best_model4.h5')


img_path = '/content/drive/MyDrive/rini_datascience/emergency/dataset/images/436.jpg'
img = image.load_img(img_path, target_size=(128,128,3))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255


predictions = model.predict(img)
predicted_class = np.argmax(predictions, axis=1)[0]
class_labels = ['non-emergency', 'emergency']
predicted_label = class_labels[predicted_class]
print('Predicted class:', predicted_label)


1/1 [==============================] - 2s 2s/step
Predicted class: non-emergency


## ResNet50V2

In [63]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [65]:
# Load pre-trained ResNet50V2 model without the top layers
base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(128,128, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 134, 134, 3)          0         ['input_6[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 64, 64, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 66, 66, 64)           0         ['conv1_conv[0][0]']          
                                                                                            

In [66]:
#Early stopping to get opimal epochs
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

ES = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 4)
MC = ModelCheckpoint('best_model2.h5', monitor = 'val_accuracy')

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(x = training_set, validation_data = validation_set, epochs = 10, callbacks = [ES, MC])

Epoch 1/10
165/165 [==============================] - 76s 437ms/step - loss: 0.4733 - accuracy: 0.8178 - val_loss: 0.3359 - val_accuracy: 0.8359
Epoch 2/10
165/165 [==============================] - 71s 429ms/step - loss: 0.3356 - accuracy: 0.8595 - val_loss: 0.2999 - val_accuracy: 0.8693
Epoch 3/10
165/165 [==============================] - 70s 427ms/step - loss: 0.2820 - accuracy: 0.8952 - val_loss: 0.2890 - val_accuracy: 0.8663
Epoch 4/10
165/165 [==============================] - 62s 379ms/step - loss: 0.2818 - accuracy: 0.8831 - val_loss: 0.3672 - val_accuracy: 0.8389
Epoch 5/10
165/165 [==============================] - 71s 429ms/step - loss: 0.2478 - accuracy: 0.8990 - val_loss: 0.3048 - val_accuracy: 0.8815
Epoch 6/10
165/165 [==============================] - 63s 380ms/step - loss: 0.2536 - accuracy: 0.8945 - val_loss: 0.3730 - val_accuracy: 0.8419
Epoch 7/10
165/165 [==============================] - 62s 378ms/step - loss: 0.2402 - accuracy: 0.9021 - val_loss: 0.3124 - val_ac

In [68]:
#prediction

import tensorflow as tf
from keras.preprocessing import image
import numpy as np

model = tf.keras.models.load_model('best_model2.h5')


img_path = '/content/drive/MyDrive/rini_datascience/emergency/dataset/images/436.jpg'
img = image.load_img(img_path, target_size=(128,128,3))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255


predictions = model.predict(img)
predicted_class = np.argmax(predictions, axis=1)[0]
class_labels = ['non-emergency', 'emergency']
predicted_label = class_labels[predicted_class]
print('Predicted class:', predicted_label)


1/1 [==============================] - 1s 728ms/step
Predicted class: non-emergency


In [69]:
metrics=['accuracy',
tf.keras.metrics.Precision(),
tf.keras.metrics.Recall(),
tf.keras.metrics.AUC(),
tf.keras.metrics.SensitivityAtSpecificity(0.5),
tf.keras.metrics.SpecificityAtSensitivity(0.5)]

In [70]:
metrics

['accuracy',

In [71]:
#evaluation
validation_loss, validation_accuracy = model.evaluate(validation_set)
print("Validation Accuracy:", validation_accuracy)



42/42 [==============================] - 16s 340ms/step - loss: 0.4020 - accuracy: 0.8602
Validation Accuracy: 0.8601823449134827


In [72]:
validation_loss

0.40198585391044617

# Accuracy Comparision


In [99]:
import plotly.express as px

model_names=['cnn','VGG16','Xception','DENSENET','RESNET']
accuracy_score=[79,79,87,89,86]


fig = px.bar(x=model_names, y=accuracy_score, text=np.round(accuracy_score, 2), title='Model Accuracy Comparison')
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(xaxis_title='Models', yaxis_title='Accuracy', yaxis_tickformat='%')
fig.show()